In [2]:
import ee
import geemap
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='daytime-images')

In [4]:
def maskS2clouds(image):
  qa = image.select('QA60')

  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

In [5]:
roi = ee.Geometry.Polygon([
  [-46.820288999334615, -23.42480237827473],
  [-46.82030136081531, -23.425705216164456],
  [-46.82127974640884, -23.42569380585392], 
  [-46.82126737829299, -23.424790968455305], 
  [-46.820288999334615, -23.42480237827473]
])
    
startDate = ee.Date('2022-01-01')
endDate = ee.Date('2022-12-31')

dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(roi).filterDate(startDate, endDate).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(maskS2clouds)

In [10]:
bandsToExport = ['B4', 'B3', 'B2']
image = dataset.median()

In [11]:
Map = geemap.Map(width="800px", height="500px")

# Add median image
Map.addLayer(image, {'bands': bandsToExport, 'max': 2000}, 'Median Image')

# Add AOI
Map.addLayer(roi, {
  'color': '00FF00', # Green
  'width': 2  # Outline thickness
}, 'Palgongsan National Park')

Map.centerObject(roi, 12)
Map

EEException: Image.load: Image asset 'B2' not found (does not exist or caller does not have access).

In [12]:
geemap.ee_export_image_to_drive(image,
    description='median_image2',
    folder='export',
    region=roi,
    scale=10
)

EEException: Image.load: Image asset 'B2' not found (does not exist or caller does not have access).